In [2]:
import numpy as np
import theano
import theano.tensor as T

from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

%matplotlib inline

In [51]:
def rolling_window(a, window):
    """
    Make an ndarray with a rolling window of the last dimension

    Parameters
    ----------
    a : array_like
        Array to add rolling window to
    window : int
        Size of rolling window

    Returns
    -------
    Array that is a view of the original array with a added dimension
    of size w.

    Examples
    --------
    >>> x=np.arange(10).reshape((2,5))
    >>> rolling_window(x, 3)
    array([[[0, 1, 2], [1, 2, 3], [2, 3, 4]],
           [[5, 6, 7], [6, 7, 8], [7, 8, 9]]])

    Calculate rolling mean of last dimension:

    >>> np.mean(rolling_window(x, 3), -1)
    array([[ 1.,  2.,  3.],
           [ 6.,  7.,  8.]])

    """
    assert window >= 1, "`window` must be at least 1."
    assert window < a.shape[-1], "`window` is too long."

    # # with strides
    shape = a.shape[:-1] + (a.shape[-1] - window, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [88]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def rectify(X):
    return T.maximum(X, 0.)

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def poiss(X):
    

def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

def model(X, w):
    return rectify(conv2d(X, w, border_mode='full'))

# simulate a toy conv-LN model

In [45]:
# fake filter
filter_dims = (5,5,10)
xm, ym = np.meshgrid(np.linspace(-5,5,filter_dims[0]), np.linspace(-5,5,filter_dims[1]))

z = xm**2 + ym**2
z = np.abs(z.max() - z)

f_star = np.outer(z, np.sin(np.linspace(0, 2*np.pi, filter_dims[2]))).reshape(5,5,-1)

In [81]:
# generate stimulus
num_samples = 1000
stim_size = (32,32)

stim = np.random.randn(*(stim_size + (num_samples,)))

In [82]:
stim_us = rolling_window(stim, filter_dims[2])
stim_us = np.rollaxis(np.rollaxis(stim_us, 3), 3)

In [83]:
f_conv = np.rollaxis(f_star,2)[np.newaxis,:,:,:]

In [106]:
# toy example
w = init_weights(f_star.shape)

(100,)

In [111]:
x = T.ftensor4()
y = T.fvector()

In [118]:
def sqerr(x, y):
    u = T.tensordot(w, x, ([0,1,2],[0,1,2]))
    return 0.5*((y-u)**2)

cost = T.mean(sqerr(x,y))
updates = RMSprop(cost, [w], lr=0.001)

In [ ]:
train = theano.function(inputs=[x, y], outputs=cost, updates=updates, allow_input_downcast=True)